In [1]:
from webdriver_manager.chrome import ChromeDriverManager

# install Chrome driver if not installed
driver = ChromeDriverManager().install()

AttributeError: 'NoneType' object has no attribute 'split'

In [2]:
from confluent_kafka import Producer
import json

# with open('data.json', 'w') as json_file:
#     json.dump(feed, json_file, indent=4)

# with open('data.json', 'r') as json_file:
#     data = json.load(json_file)

def produce_data(topic_name:str, data: dict):
    """
    Send data to the broker
    Args:
        data (dict): data to send
    """
    # Configure the Kafka producer
    conf = {
        'bootstrap.servers': '127.0.0.1:9092',  # Broker address
        'client.id': 'python-producer',
        'delivery.timeout.ms': 5000
    }

    # Create a Kafka producer with the given configuration
    producer = Producer(conf)

    # Callback function to handle delivery reports
    def delivery_report(err, msg):
        if err is not None:
            print(f"Message delivery failed: {err}")
        else:
            print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

    # Send the JSON data to the Kafka topic
    producer.produce(topic_name, value=json.dumps(data), on_delivery=delivery_report)
    producer.poll(10000)

    # Wait for any outstanding messages to be delivered and delivery reports to be received
    producer.flush()

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import json
from datetime import datetime

options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

url = 'https://www.cryptocompare.com/coins/list/all/USD/1'

# start connection
driver.get(url)
trs = driver.find_elements(By.CSS_SELECTOR, "tr.ng-scope")

currencies = []
try:
    for i, tr in enumerate(trs):
        if i == 10:
            break

        tds = tr.find_elements(By.CSS_SELECTOR, "td")

        currency = {
            "place": tds[0].find_element(By.XPATH, "div").text,
            "name": tds[2].find_element(By.CSS_SELECTOR, "h3").text,
            "price": tds[3].find_element(By.XPATH, "div").text,
            "volume": tds[5].find_element(By.XPATH, "div").text,
            "top_tier_volume": tds[6].find_element(By.XPATH, "div").text,
            "market_cap": tds[7].find_element(By.XPATH, "div").text,
            "percentage_change": tds[9].find_element(By.XPATH, "div").text
        }

        currencies.append(currency)

    # save the informations to json
    feed = {
        "timestamp": str(datetime.now()),
        "data": currencies
    }

    # with open('data.json', 'w') as json_file:
    #     json.dump(feed, json_file, indent=4)
    # produce the data to the broker
    produce_data('crypto.cryptocompare', feed)
    
    
except Exception as error:
    raise(error)

# close the connection
driver.quit()

delete topics (it is crashing in confluent admin)

In [18]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions, ConfigResource

conf = {
    'bootstrap.servers': '127.0.0.1:9092',  # Broker address
}

admin_client = AdminClient(conf)

topic_to_delete = "y"
admin_client.delete_topics([topic_to_delete], operation_timeout=30)

print(f"Topic {topic_to_delete} deleted.")

Topic y deleted.
